In [39]:
# %load_ext autoreload
%autoreload 2
import torch
from recall import Recall
from datasets import TripletAudio

K, MAX_CLOSE_NEG, MAX_FAR_NEG = 5, 15, 15
BATCH_SIZE = 128

triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
 = TripletAudio(False, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [157]:
%autoreload 2
# Set up the network and training parameters
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from recall import Recall
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT, RECALL_NUM_CAND = 0.5, 1e-3, 10, 100, 25

#define model
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)

loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

recall = Recall(RECALL_NUM_CAND, K)

initialising model biases
done


In [158]:
%autoreload 2
from trainer import fit
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)

Train: [0/48048 (0%)]	Loss: 33.801556
Train: [12800/48048 (27%)]	Loss: 18.444141
Train: [25600/48048 (53%)]	Loss: 17.540117
Train: [38400/48048 (80%)]	Loss: 13.794197
Epoch: 1/10. Train set: Average loss: 15.6334
Epoch: 1/10. Validation set: Average loss: 18.9504
Train: [0/48048 (0%)]	Loss: 10.866055
Train: [12800/48048 (27%)]	Loss: 9.506681
Train: [25600/48048 (53%)]	Loss: 8.078267
Train: [38400/48048 (80%)]	Loss: 6.763571
Epoch: 2/10. Train set: Average loss: 7.6689
Epoch: 2/10. Validation set: Average loss: 9.0134
Train: [0/48048 (0%)]	Loss: 6.106470
Train: [12800/48048 (27%)]	Loss: 4.939356
Train: [25600/48048 (53%)]	Loss: 4.379292
Train: [38400/48048 (80%)]	Loss: 3.775523
Epoch: 3/10. Train set: Average loss: 4.2366
Epoch: 3/10. Validation set: Average loss: 5.5188
Train: [0/48048 (0%)]	Loss: 5.372194
Train: [12800/48048 (27%)]	Loss: 3.293025
Train: [25600/48048 (53%)]	Loss: 3.118892
Train: [38400/48048 (80%)]	Loss: 2.740443
Epoch: 4/10. Train set: Average loss: 2.9650
Epoch: 4/10

In [181]:
# perform recall on random batch
outputs = []
for d in (triplet_train_dataset, triplet_test_dataset):
    batch_indicies = np.random.choice(list(range(0, d.get_dataset().shape[0])), 1000, False)
    queries = d.get_dataset()[batch_indicies]
    true_knns = d.get_KNN().iloc[batch_indicies, :]
    outputs.append(recall.calculate(d.get_dataset(), model.embedding_net, queries, true_knns, False))
outputs

embedding dataset
quantizing dataset
done quantizing into 5636 buckets
embedding dataset
quantizing dataset
done quantizing into 1191 buckets


[array([0.8, 0. , 0.4, 0.2, 0.2, 0. , 0.8, 0.6, 0.6, 0. , 0. , 0. , 0.8,
        0. , 0.4, 0.8, 0.2, 0.2, 0.2, 0. , 0.2, 0. , 0.4, 0. , 0.2, 0.2,
        0.2, 0.8, 0.4, 0.6, 0.8, 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0.8, 0.2, 0. , 0. , 0.6, 0. , 0.2, 0. , 0. , 0. , 0. , 0.8,
        0. , 0.8, 0.8, 0. , 0. , 0. , 0.4, 0.4, 0. , 0.2, 0. , 0. , 0.2,
        0.2, 0. , 0. , 0.2, 0.8, 0. , 0. , 0.2, 0. , 0. , 0.4, 0. , 0.8,
        0. , 0. , 0.4, 0. , 0.2, 0. , 0. , 0.2, 0.8, 0.8, 0.6, 0.6, 0.4,
        0.8, 0.2, 0.8, 0. , 0.8, 0.2, 0.2, 0. , 0. , 0.2, 0.8, 0.6, 0.2,
        0.2, 0. , 0. , 0.8, 0.2, 0.4, 0.4, 0. , 0.2, 0.8, 0.6, 0. , 0. ,
        0.2, 0.2, 0. , 0. , 0. , 0.2, 0.8, 0.8, 0.6, 0.8, 0. , 0.8, 0.2,
        0.4, 0. , 0.4, 0.8, 0.8, 0.8, 0. , 0. , 0. , 0. , 0. , 0.6, 0.8,
        0.2, 0.4, 0.8, 0.2, 0. , 0. , 0.8, 0. , 0. , 0.2, 0.4, 0.2, 0.6,
        0. , 0. , 0.8, 0. , 0.8, 0.4, 0. , 0.2, 0.4, 0.2, 0.6, 0.2, 0.8,
        0.8, 0.2, 0. , 0.4, 0.2, 0. , 0. , 0.8, 0.2

### Online Selection

In [24]:
# %load_ext autoreload
%autoreload 2
import torch
from datasets import AudioTrainDataset, AudioTestDataset
from datasets import BalancedBatchSampler

K = 5
train_dataset = AudioTrainDataset(K)
test_dataset = AudioTestDataset(K)

train_batch_sampler = BalancedBatchSampler(train_dataset)
test_batch_sampler = BalancedBatchSampler(test_dataset)

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler)

In [33]:
# Set up the network and training parameters
from networks import EmbeddingNet, AnchorNet
from losses import OnlineTripletLoss
from utils import SemihardNegativeTripletSelector, HardestNegativeTripletSelector
from metrics import AverageNonzeroTripletsMetric
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 5, 50

#define model
anchor_net = AnchorNet(train_dataset.data, INPUT_D, OUTPUT_D)
model = EmbeddingNet(anchor_net)
loss_fn = OnlineTripletLoss(MARGIN, SemihardNegativeTripletSelector(MARGIN, train_dataset.KNN))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

initialising model biases
done
generating similarity matrix
done


In [36]:
%autoreload 2
from trainer import fit
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT, metrics=[AverageNonzeroTripletsMetric()])

TypeError: forward() missing 1 required positional argument: 'indicies'